# `transformnd` tutorial

`transformnd` is a python library which provides implementations of coordinate transformations, base classes to ensure a consistent API across implementations, and utilities for combining and working with those coordinate transformations.

First, define a basic shape, and make a function for plotting it:

In [ ]:
%matplotlib inline
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.patches import Polygon

rng = np.random.default_rng(1991)

# Array of N coordinates in D dimensions has shape NxD
square = np.array(
    [
        [0, 0],
        [0, 1],
        [1, 1],
        [1, 0],
    ]
)


def plot_shape(**kwargs):
    """Draw a labelled polygon from kwargs like name=coords."""
    ax = plt.gca()
    min_ = -1
    max_ = 3
    ax.set_xlim(min_, max_)
    ax.set_ylim(min_, max_)
    ax.hlines(0, min_, max_, colors="k")
    ax.vlines(0, min_, max_, colors="k")
    ax.set_aspect("equal")
    sq = Polygon(square, closed=False, color="grey", label="original")
    ax.add_patch(sq)

    cmap = plt.get_cmap("tab10")

    for idx, (name, coords) in enumerate(kwargs.items()):
        poly = Polygon(coords, closed=False, label=name, color=cmap(idx))
        ax.add_patch(poly)
    plt.legend()


plot_shape()

Now we'll add some basic transformations.

Transformations are subclasses of the `Transform` abstract base class.
Once the transformation is instantiated, it can be used just like a function, transforming a numpy array of coordinates into a new numpy array.


In [ ]:
from transformnd.transforms.simple import Translate, Scale

translate = Translate([1.5, 0])
scale = Scale([0.5, 2])

plot_shape(
    translated=translate(square),
    scaled=scale(square),
)

## `TransformSequence`

Transforms can be chained together into a single transformation, with the `TransformSequence` class.

In [ ]:
from transformnd import TransformSequence

scale_then_translate = TransformSequence([scale, translate])

plot_shape(
    translated=translate(square),
    scaled=scale(square),
    scaled_then_translated=scale_then_translate(square),
)

## Composing and inverting transformations

Certain transformations can be inverted with the `~` operator.

`TransformSequences` can also be constructed with the `|` operator.

In [ ]:
translate_scale = translate | scale
# Note that translate -> scale gives different results to scale -> translate above
# because the scaling is done around (0, 0)

undo_translate_scale = ~translate_scale

plot_shape(
    translated_scaled=translate_scale(square),
    undone=undo_translate_scale(translate_scale(square)),
)

## Affine transformations

A common class of coordinate transformations are affine transformations.
These require a transformation matrix which then multiplies the coordinate array.
`transformnd` provides convenience methods for constructing this transformation matrix out of common operations: translations, scales, reflections, rotations, and shears.

Because this all uses matrix multiplication under the hood, `AffineTransform` instances can be composed using the matrix multiplication operator `@` rather than needing to apply the transforms one at a time in a `TransformSequence`.

Affine transformations are invertible.

In [ ]:
from transformnd.transforms.affine import AffineTransform

tran = AffineTransform.translation([1, 0])
rot = AffineTransform.rotation2(45)  # Only 2D and 3D are implemented
sca = AffineTransform.scaling([1.5, 1])
tran_rot_sca = tran @ rot @ sca

plot_shape(
    translated=tran(square),
    rotated=rot(square),
    scaled=sca(square),
    translated_rotated_scaled=tran_rot_sca(square),
)

## Spaces and bridging transforms

A common task in coordinate transformation is to convert coordinates in one space (e.g. a pixel index of an image) into some other space (e.g. a location in "world" space, using the image's resolution and offset).

All `Transform`s can take `source_space` and `target_space` keyword arguments on instantiation, which can be anything hashable and non-None (e.g. a string or number).
This makes it easier to reason about what space the coordinates start and end in.

`TransformSequence` instances check that consecutive transforms in the sequence refer to consistent spaces (if spaces are defined).
They will also use a transform with a defined target space to infer the source space of the next transform, if undefined.

Once you have a set of transforms between different defined spaces, you can do bridging transforms: calculating how to get from one space to another by applying some subset of those transforms in sequence.
This uses the `TransformGraph` class (requires `networkx`).

The `TransformGraph` will automatically unpack any transforms inside sequences (if they have spaces defined), and infer inverse transforms where possible.

In [ ]:
from transformnd.graph import TransformGraph

g = TransformGraph()
ab = Scale(2, source_space="a", target_space="b")
bc = Translate([0.5, 1], source_space="b", target_space="c")
bd = Translate([1, 0.5], source_space="b", target_space="d")

g.add_transforms([ab, bc, bd])

print("Transform sequence from a to c:", g.get_sequence("a", "c"))

print("Transform sequence from d to a:", g.get_sequence("d", "a"))

## Making your own `Transform`

There are many different types of transformation, which have been implemented in many different ways in many different libraries.
`transformnd` makes it easy to bring lots of different implementations under one interface, which can then make use of the same composition tools.

`Transform` is an abstract base class with a number of utility methods already implemented.
At a minimum, you just need to subclass it and implement the `.__call__` method.
If your transformation is only defined for certain dimensionalities, that can also be specified: use the `._validate_coords` method to check that passed-in coordinates are of the right shape. [1]

You can also optionally implement `.__invert__`, if the transformation is invertible.

[1]: N.B. `TransformSequence` and `TransformGraph` both ensure that their component transforms have compatible dimensionalities.

In [ ]:
from transformnd import Transform


class IsotropicScale2d(Transform):
    ndim = {2}  # a set of valid dimensionalities

    def __init__(self, factor: float, *, source_space=None, target_space=None):
        # ensure the spaces are handled properly
        super().__init__(source_space=source_space, target_space=target_space)
        self.factor = factor

    def __call__(self, coords: np.ndarray) -> np.ndarray:
        # ensure the coordinates are of the right shape
        self._validate_coords(coords)
        return coords * self.factor

    def __invert__(self):
        return IsotropicScale2d(
            1 / self.factor,
            source_space=self.target_space,
            target_space=self.source_space,
        )

## Adapters

Lots of objects have coordinates, but many other features too.
`transformnd` makes it easy to transform arbitrary python objects like this using adapters.
Basically, you define a subclass of `transformnd.adapters.BaseAdapter` and implement `.__call__`, which takes a transform and an object.
This method will get coordinates from the object, and construct a new one from the transformed coordinates and the old object.

For cases where an object has transformable attributes, there is the `AttrAdapter`, which takes key-value pairs of the name of the transformable attribute, and an adapter which knows how to apply a transform to that attribute.
Cases where the attribute is just a coordinate array can use the `NullAdapter`.

If you're going to use the same adapter-transform combination a lot, you can use `Adapter.partial` to create a callable representing that combination.

When writing your own `Adapter`, consider giving the `__call__` method an `in_place` keyword argument if you have the option of modifying the existing object rather than creating a new one.
`AttrAdapter`s which compose over other adapters will save themselves some copying by using this argument intelligently.

In [ ]:
from dataclasses import dataclass
from transformnd.adapters import AttrAdapter, NullAdapter


@dataclass
class Town:
    houses: np.ndarray
    shops: np.ndarray


my_town = Town(
    houses=rng.random((5, 2)),
    shops=rng.random((3, 2)),
)

town_adapter = AttrAdapter(houses=NullAdapter(), shops=NullAdapter())
translated_town = town_adapter(translate, my_town)

town_translator = town_adapter.partial(translate)
translated_town_2 = town_translator(translated_town)

`transformnd` includes adapters for instances of `meshio.Mesh`, `pandas.DataFrame` (where certain columns make up a coordinate array), and geometries from `shapely`.

In [ ]:
from transformnd.adapters.pandas import DataFrameAdapter
import pandas as pd

df = pd.DataFrame(
    np.random.random((3, 5)),
    columns=["x", "y", "z", "radius", "weight"],
)
print("original")
print(df)
xyz_adapter = DataFrameAdapter(["x", "y", "z"])
scale3d = Scale([1, 10, 100])
print("\nscaled")
print(xyz_adapter(scale3d, df))